In [1]:
import pickle
from lib.utility import ProcessPipeline
from lib.nlp import entityPipeline,featurePipeline
import pandas as pd
import numpy as np

# Part 1: data engineering

In [3]:
### read pickle file
with open('outputs/step2_news_raw.pickle', 'rb') as handle:
    texts = pickle.load(handle)

### Input: list of string

In [4]:
len(texts)

2988

In [5]:
processPipe = ProcessPipeline(texts=texts,steps=["langdetection","summarization",'tokenization'],
                           tokenization_steps=['remove_digits','remove_punctuation',"remove_stopwords",'lemmatization','stemmization'],stopwordsWhiteList=["n't", "not", "no"])

In [6]:
tokens = processPipe.run(workers=6,return_str=False)

# Part 2: Topic Modeling

In [7]:
tokens[0][:10]

['forum',
 'address',
 'one',
 'press',
 'issu',
 'lifetim',
 'global',
 'energi',
 'climat',
 'chang']

In [9]:
from lib.nlp import LDA

In [10]:
%%capture
LDAmodel = LDA(texts=tokens,num_topics=3)

In [11]:
LDAmodel.plot()

/Users/zili/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.130369  0.123531       1        1  41.762718
0      0.312595 -0.012946       2        1  34.854095
1     -0.182226 -0.110585       3        1  23.383190, topic_info=     Category         Freq        Term        Total  loglift  logprob
term                                                                 
141   Default  3189.000000        hous  3189.000000  30.0000  30.0000
2039  Default  2242.000000       trump  2242.000000  29.0000  29.0000
806   Default  2385.000000    democrat  2385.000000  28.0000  28.0000
2784  Default  1720.000000     speaker  1720.000000  27.0000  27.0000
3347  Default  1698.000000      pelosi  1698.000000  26.0000  26.0000
27    Default  2542.000000      energi  2542.000000  25.0000  25.0000
400   Default  1349.000000    congress  1349.000000  24.0000  24.0000
263   Default   860.000000       share   860.000000  23.0000  23.0000
978   Default  1986.000000      govern  1986.000000  22.0000  22.0000
3336  Default   994.000000       senat   994.000000  21.0000  21.0000
85    Default  1086.000000     compani  1086.000000  20.0000  20.0000
891   Default  1361.000000      leader  1361.000000  19.0000  19.0000
174   Default   822.000000      market   822.000000  18.0000  18.0000
3344  Default   814.000000       white   814.000000  17.0000  17.0000
1861  Default   798.000000        wall   798.000000  16.0000  16.0000
67    Default   916.000000       solar   916.000000  15.0000  15.0000
992   Default   834.000000  republican   834.000000  14.0000  14.0000
150   Default  4181.000000         new  4181.000000  13.0000  13.0000
922   Default  1618.000000           —  1618.000000  12.0000  12.0000
2197  Default   837.000000       parti   837.000000  11.0000  11.0000
3026  Default   667.000000      border   667.000000  10.0000  10.0000
13    Default   874.000000      climat   874.000000   9.0000   9.0000
3083  Default   632.000000         rep   632.000000   8.0000   8.0000
2313  Default   708.000000     histori   708.000000   7.0000   7.0000
3337  Default   616.000000    shutdown   616.000000   6.0000   6.0000
1338  Default   918.000000      return   918.000000   5.0000   5.0000
2461  Default   579.000000       nanci   579.000000   4.0000   4.0000
267   Default   409.000000       stock   409.000000   3.0000   3.0000
1331  Default   606.000000       readi   606.000000   2.0000   2.0000
104   Default   950.000000       renew   950.000000   1.0000   1.0000
...       ...          ...         ...          ...      ...      ...
572    Topic3   148.831833      applic   167.138870   1.3371  -6.1391
601    Topic3   168.286163      revenu   192.391068   1.3193  -6.0163
254    Topic3   442.028320        rate   571.422119   1.1964  -5.0506
735    Topic3   103.105576      letter   111.694748   1.3731  -6.5062
85     Topic3   726.523254     compani  1086.342041   1.0509  -4.5537
67     Topic3   624.672302       solar   916.234253   1.0701  -4.7047
234    Topic3   182.647232        hold   218.026779   1.2761  -5.9344
27     Topic3  1364.774292      energi  2542.589111   0.8310  -3.9232
258    Topic3   420.896179      report   610.516174   1.0812  -5.0996
1574   Topic3   168.725540      storag   204.691086   1.2599  -6.0137
387    Topic3   114.213570       asset   126.990799   1.3471  -6.4039
571    Topic3   228.348999       addit   317.602844   1.1232  -5.7111
252    Topic3   225.728882       price   323.916840   1.0920  -5.7226
104    Topic3   488.826538       renew   950.348145   0.7883  -4.9500
194    Topic3   230.197708       manag   335.077209   1.0777  -5.7030
376    Topic3   575.859619       state  1365.518799   0.5897  -4.7861
111    Topic3   356.803741        wind   677.084534   0.8125  -5.2648
192    Topic3   273.940948      invest   459.027893   0.9370  -5.5291
459    Topic3   365.178925     project   730.465820   0.7599  -5.2416
64     Topic3   231.741699    research   368.00

In [13]:
### Check each document and its topics
topicDic = {0:[],1:[],2:[]}
for i in range(len(LDAmodel.corpus)):
    allTopics = LDAmodel.get_doc_topics(docId=i,print_topics=False) ### eg: [(1, 0.58676654), (2, 0.4099236)]
    for topic in allTopics:
        topicID = topic[0]
        prob = topic[1]
        if prob > 0.6:
            topicDic[topicID] += [i]
            break

In [14]:
topicDic.keys()

dict_keys([0, 1, 2])

# Part 3: Sentiment analysis

In [18]:
summarizePipe = ProcessPipeline(texts=texts,steps=["langdetection","summarization"])
summarized = summarizePipe.run()

In [14]:
summarized[0]

'The forum addressed one of the most pressing issues of our lifetimes - global energy and climate change.\nIndia’s development will undoubtedly be fuelled by an increase in energy consumption, but this economic development belies a growing problem - climate change caused by CO2 emissions.\nThis includes increase in Earth’s mean surface temperature (also known as global warming), rise in sea level and acidification, extreme weather events, and so on.\nWhile the increase in global temperature and loss of polar ice has been strongly linked to anthropogenic activities (particularly CO2 emissions), there is no consensus among researchers about the link between extreme weather events such as forest fires, cyclones, droughts etc and anthropogenic causes.\nNow, concerning the timeline, it is expected that global temperatures will increase by over 2 degrees C by 2040 if emissions continue as before, well within our lifetimes for most of us reading this article.\nMillions of Indians live off the

In [19]:
featurePipe = featurePipeline(texts=summarized)
featureDic = featurePipe.run()

In [21]:
sentDF = pd.DataFrame(featureDic)
sentDF.head()

,compound,neg,neu,polarity,pos,subjectivity
0,0.9178,0.045,0.828,0.079527,0.127,0.403542
1,0.0000,0.000,0.000,0.000000,0.000,0.000000
2,0.0000,0.000,0.000,0.000000,0.000,0.000000
3,0.9468,0.000,0.778,0.466667,0.222,0.766667
4,0.9584,0.011,0.818,0.052548,0.172,0.523140


In [17]:
len(featureDic)

2988

In [18]:
posFeatures = sentDF['pos'].values
negFeatures = sentDF['neg'].values

In [19]:
featureDic = {}
for topicID in topicDic:
    newsIDs = [topicDic[topicID]]
    
    tmpDic = {'pos':posFeatures[newsIDs],'neg':negFeatures[newsIDs]}
    featureDic[topicID] = tmpDic

/Users/zili/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  """


In [21]:
top3Dic = {}
for topicID in featureDic:
    tmpPos = featureDic[topicID]['pos']
    tmpNeg = featureDic[topicID]['neg']
    
    top3pos = np.argsort(tmpPos)[-3:]
    top3neg =  np.argsort(tmpNeg)[-3:]
    
    tmpDic = {}
    tmpDic['pos'] = top3pos[::-1]
    tmpDic['neg'] = top3neg[::-1]
    
    top3Dic.update({topicID:tmpDic})

np.argsort(featureDic[1]['pos'])
np.sort(featureDic[1]['pos'])

In [22]:
top3Dic

{0: {'pos': array([931, 314, 105]), 'neg': array([916, 421, 169])},
 1: {'pos': array([110, 107,  46]), 'neg': array([  1, 110, 236])},
 2: {'pos': array([413, 128, 276]), 'neg': array([349, 347, 391])}}

In [23]:
for topicID in top3Dic:
    print(f"For topic {topicID}: three most positive articles are : {top3Dic[topicID]['pos']}, three most negative articles are : {top3Dic[topicID]['neg']}")

For topic 0: three most positive articles are : [931 314 105], three most negative articles are : [916 421 169]
For topic 1: three most positive articles are : [110 107  46], three most negative articles are : [  1 110 236]
For topic 2: three most positive articles are : [413 128 276], three most negative articles are : [349 347 391]


# Part 4: Keyword extraction

In [24]:
entityDic = {}

In [25]:
for topicID in topicDic:
    includedNewsIndex = topicDic[topicID]
    includedNews = np.array(summarized)[includedNewsIndex]
    print("Start to process of topic:",topicID)
    pipeline = entityPipeline(texts=includedNews.tolist(),targetPOSs=[]) ### only interested in entity
    entities = pipeline.run()
    entityDic[topicID] = entities
    print("Finish process of topic:",topicID)

Start to process of topic: 0
Finish process of topic: 0
Start to process of topic: 1
Finish process of topic: 1
Start to process of topic: 2
Finish process of topic: 2


In [26]:
entityDic[1]

[{'Trump': {'count': 1, 'label': 'PERSON'}},
 {},
 {'Washington': {'count': 1, 'label': 'GPE'},
  'Democrats': {'count': 3, 'label': 'NORP'},
  'Nancy Pelosi': {'count': 1, 'label': 'PERSON'},
  'House': {'count': 4, 'label': 'ORG'},
  '116th Congress': {'count': 1, 'label': 'ORG'},
  'Donald Trump': {'count': 1, 'label': 'PERSON'},
  'America': {'count': 1, 'label': 'GPE'},
  'Pelosi': {'count': 4, 'label': 'PERSON'},
  'first': {'count': 2, 'label': 'ORDINAL'},
  'Congress': {'count': 3, 'label': 'ORG'},
  'Americans': {'count': 1, 'label': 'NORP'},
  'Trump': {'count': 2, 'label': 'NORP'},
  'the White House': {'count': 2, 'label': 'ORG'},
  "Trump's": {'count': 2, 'label': 'ORG'},
  'U.S': {'count': 1, 'label': 'GPE'},
  'Mike Pence': {'count': 1, 'label': 'PERSON'},
  'Senate': {'count': 1, 'label': 'ORG'},
  'Republicans': {'count': 1, 'label': 'NORP'},
  'Mitch McConnell': {'count': 1, 'label': 'PERSON'},
  'New Yorker': {'count': 1, 'label': 'ORG'},
  'Ocasio-Cortez': {'count':